# Model comparison after Federated Learning

In [1]:
import sys
sys.path.insert(0, "../")

In [2]:
import json
import pandas as pd

from decentralized_smart_grid_ml.federated_learning.models_reader_writer \
    import load_fl_model, load_fl_model_weights

2021-10-25 18:28:54.417735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-25 18:28:54.417767: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
task_config_path = "../data_sample/simple_ml_task/simple_ml_task_config.json"
with open(task_config_path, "r") as file_read:
    json_config = json.load(file_read)

In [4]:
N_PARTICIPANTS = 2
task_name = json_config["task_name"]

model_path = json_config["baseline_model_artifact"]
final_model_weights = "../data_sample/simple_ml_task/validator/validator_weights_final.json"
features_names = json_config["features_names"]
participants_directory = "../data_sample/" + task_name + "/participants/participant_" 
test_set_path = "../data_sample/simple_ml_task/validator/" + task_name + "_test.csv"

Load the participants' datasets and concatenate them to obtain a single dataset

In [5]:
participants_datasets = []
for i in range(N_PARTICIPANTS):
    participants_datasets.append(
        pd.read_csv(
            participants_directory + str(i) + "/" + task_name + "_" + str(i) + ".csv"
        )
    )
whole_participants_df = pd.concat(participants_datasets)

In [6]:
x_train = whole_participants_df[features_names["features"]].values
y_train = whole_participants_df[features_names["labels"]].values

Load the test set used to compute the metrics

In [7]:
test_set_df = pd.read_csv(test_set_path)
x_test = test_set_df[features_names["features"]].values
y_test = test_set_df[features_names["labels"]].values

Load and train the original model with the participants' datasets

In [8]:
model = load_fl_model(model_path)
model.fit(x_train, y_train, epochs=json_config["epochs"])

2021-10-25 18:28:56,542 - decentralized_smart_grid_ml.federated_learning.models_reader_writer - INFO - Load model from  /home/fabsam/Documenti/university/masterDegree/thesis/Decentralized-SmartGrid-ML/data_sample/simple_ml_task/linear_model


2021-10-25 18:28:56.391756: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-25 18:28:56.391789: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-25 18:28:56.391815: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Lenovo-ideapad): /proc/driver/nvidia/version does not exist
2021-10-25 18:28:56.392059: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-25 18:28:56.593549: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR

Epoch 1/2
13/13 [==============================] - 0s 1ms/step - loss: 0.2087 - accuracy: 0.8925
Epoch 2/2
13/13 [==============================] - 0s 2ms/step - loss: 0.1420 - accuracy: 0.9975


Load the global model obtained with the federated learning approach

In [9]:
fl_model = load_fl_model(model_path)
final_weights = load_fl_model_weights(final_model_weights)
fl_model.set_weights(final_weights)
fl_model.evaluate(x_test, y_test)

2021-10-25 18:28:57,177 - decentralized_smart_grid_ml.federated_learning.models_reader_writer - INFO - Load model from  /home/fabsam/Documenti/university/masterDegree/thesis/Decentralized-SmartGrid-ML/data_sample/simple_ml_task/linear_model
2021-10-25 18:28:57,178 - decentralized_smart_grid_ml.federated_learning.models_reader_writer - INFO - Loaded model's weights from ../data_sample/simple_ml_task/validator/validator_weights_final.json
4/4 [==============================] - 0s 2ms/step - loss: 0.1094 - accuracy: 1.0000


[0.10936731100082397, 1.0]

Compare the results

In [10]:
print("Model trained with the participants' dataset: ", model.evaluate(x_test, y_test))
print("Global model trained with the FL approach: ", fl_model.evaluate(x_test, y_test))

4/4 [==============================] - 0s 1ms/step - loss: 0.1105 - accuracy: 1.0000
Model trained with the participants' dataset:  [0.11047482490539551, 1.0]
4/4 [==============================] - 0s 1ms/step - loss: 0.1094 - accuracy: 1.0000
Global model trained with the FL approach:  [0.10936731100082397, 1.0]
